## 1.Importing the dependencies

In [1]:
import os
import sys
import nltk
import pandas as pd
import numpy as np
import glob
import json
from tqdm import tqdm

In [ ]:
# !pip install sentence-transformers

In [ ]:
# Loading Model and tokenizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
sbert_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2').to("cuda")

In [6]:
dataset = "IN-Abs"

In [7]:
def get_root_path():
    #function to get root path of dataset
    path = "/kaggle/input/dataset/"
    return path

In [ ]:
def get_summary_data(dataset,folder):
    #function to get names, documents, and summaries
    path = get_root_path() + dataset + '/' + folder + '/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + dataset + '/' + folder + '/summary'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f:
            a = f.read()
            l = len(a)
            data_summary.append(a)

    return names, data_source, data_summary

In [ ]:
#Reading the documents and summaries 
names, data_source, data_summary = get_summary_data(dataset, "train-data")
print(len(names))
print(len(data_source))
print(len(data_summary))

In [ ]:
def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)
    return sents

In [ ]:
def similarity_l_l(l1, l2):
    '''
    Function to find the most similar sentence in the document for each sentence in the summary 
    input:  l1 - Summary sentences
            l2 - Document sentences
    returns a list of document sentence indexes for each sentence in the summary 
    '''
    document_embeddings = sbert_model.encode(l1+l2)
    similarities=cosine_similarity(document_embeddings)
    
    result = []
    for i in range(len(l1)):
        vals = similarities[i]
        vals = vals[len(l1):]
        idx = np.argmax(vals)
        result.append(idx)
    return result

In [ ]:
def nest_sentencesV2(document,chunk_length):
    '''
    function to chunk a document
    input:  document           - Input document
            chunk_length        - chunk length
    output: list of chunks. Each chunk is a list of sentences.
    '''
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(sent)
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(sent)
    return nested

In [ ]:
def get_chunks_data_from_docV2(doc, summ):
    '''
    Function to generate chunks along with their summaries 
    input:  doc - legal Document
            summ - Gold standard summary
    returns a list of chunks and their summaries 
    '''
    chunk_summ_word_threshold = 150
    sentence_mapping = {}
    doc_sents = split_to_sentences(doc)
    summ_sents = split_to_sentences(summ)
    
    result = (similarity_l_l(summ_sents,doc_sents))
    
    for i in range(len(summ_sents)):
        sentence_mapping[doc_sents[result[i]]] = summ_sents[i]
    
    final_chunks = []
    final_summ = []
    for chunk in nest_sentencesV2(doc, 1024):
        summ = ""
        for chunk_sent in chunk:
            if chunk_sent in sentence_mapping:
                summ = summ + sentence_mapping[chunk_sent]
        if len(summ.split(" ")) >= chunk_summ_word_threshold:
            final_chunks.append(" ".join(chunk))
            final_summ.append(summ)
    return final_chunks, final_summ


In [ ]:
#loop to pass every document, generate the fine tuning data and saving in a excel file 
import pandas as pd
training_chunks = []
training_summs = []
for i in tqdm(range(len(data_source))):
    cks, summs = get_chunks_data_from_docV2(data_source[i],data_summary[i])
    training_chunks = training_chunks + cks
    training_summs = training_summs + summs
#     print(i, len(training_summs), end = ", ", sep = " : ")
    if i%100 == 0: 
        full = list(zip(training_chunks,training_summs))
        df = pd.DataFrame(full,columns=['data', 'summary'])
        df.to_excel("FD_" + dataset + "_CSM_BK_512.xlsx")
#         break
full = list(zip(training_chunks,training_summs))
df = pd.DataFrame(full,columns=['data', 'summary'])
df.to_excel("FD_" + dataset + "_CSM_512.xlsx")